# Convolutional Neural Network

### Importing the libraries

The ImageDataGenerator will be used to preprocess images

In [13]:
import tensorflow as tf 
import graphviz
import pydot
from keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras

In [2]:
tf.__version__

'2.3.1'

## Part 1 - Data Preprocessing

Preprocessing of Data for only Training Sets to avoid overfitting. We would apply some transformations eg rotations, zooms, rotation on our images to get them augumented so the CNN doesn't overlearn on the particular image we have.
<br>

We have Image, timeseries and text preprocessing. We can find details on the keras preprocessing page. We can use some of the default transformations.  

shear_range: Float. Shear Intensity (Shear angle in counter-clockwise direction in degrees)
<br>
zoom_range: Float or [lower, upper]. Range for random zoom. 
<br>
horizontal_flip: Boolean. Randomly flip inputs horizontally.


### Preprocessing the Training set

In [3]:
train_datagen = ImageDataGenerator(
        rescale=1./255,  # Feature scaling to all pixels, normalization of our pixels since they are btw 0-255
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# connect to the training set 
train_set = train_datagen.flow_from_directory(
        'dataset/training_set',   # training set 
        target_size=(64, 64),  #final size of the images, the smaller the better 
        batch_size=32,           # images per batch 
        class_mode='binary')     # class mode, since it's cat or dogs we leave it at binary or categorical 

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [4]:
test_datagen = ImageDataGenerator(rescale=1./255) # 
test_set = test_datagen.flow_from_directory(
        'dataset/test_set',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [5]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [6]:
#add first layer 
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64,64,3]))
# filters are the features detectors, kernel_size i.e 3X3 matrix 
## input shape of your input, our input shape would be 64,64 since we reduced it, 3 for colored 

### Step 2 - Pooling
Getting the maximum pixel of the featured map matrix to be pooled. <br> 
pool_size indicates the size of the array which is mostly 2. <br>
The slides would be the movement of the matrix which would be 2. 


In [7]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [8]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening
Flattening all to become a one dimensional vector which would be the input

In [9]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [10]:
#full connected layer 
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [11]:
# one neuron since we have a binary classifier, if more we use the softmax activation
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [14]:
keras.utils.plot_model(model, "cnn_plot.png", show_shapes=True)

NameError: name 'model' is not defined

## Part 3 - Training the CNN

### Compiling the CNN

In [20]:
# loss biniary, metrics can be multiple, accuracy most useful in cnn 
cnn.compile(optimizer = 'adam', loss='binary_crossentropy', metrics= ['accuracy', 'mse'])

### Training the CNN on the Training set and evaluating it on the Test set

In [22]:
cnn.fit(x = train_set, validation_data = test_set, epochs=1)

250/250 [==============================] - 139s 558ms/step - loss: 0.6890 - acc: 0.5257 - mean_squared_error: 0.2480 - val_loss: 0.6854 - val_acc: 0.5575 - val_mean_squared_error: 0.2461


## Part 4 - Making a single prediction

In [35]:
import numpy as np 
#
from keras.preprocessing import image 
test_image = image.load_img('dataset/single_prediction/cat_or_dog_2.jpg', target_size=(64,64))

#convert to 2D array, predict method expects an array
test_image = image.img_to_array(test_image)
#cnn was trained on batches of 32, the single image also has to be in the batch 
#  We would be adding  a dimension to the axis 0 the dimension of the batch would be the first dimension
test_image = np.expand_dims(test_image, axis = 0 )
result  = cnn.predict(test_image)
# encoding, we need to figure out which is zero and one, from the structure of the folders 
train_set.class_indices
if result[0][0] == 1:
    prediction = 'dog'
else:
    prediction = 'cat'

In [36]:
print(prediction)

cat
